In [1]:
import sys
import matplotlib.pyplot as plt


#sys.path.append("../C_20_data")
#sys.path.append("../thesis-carbon-folding/Unfolding")
#sys.path.append("../thesis-carbon-folding/functions")
#sys.path.append("../thesis-carbon-folding/Unfolding/Application_functions")
#sys.path.append("../data/")
sys.path.append("../unfolding/")

from Application_functions import read_unfolding
from read_geometries import read_geometry
from geometry_functions import *

dual_unfolding, graph_unfolding, graph_unfolding_faces, vertices_final, bonds_toBe, lengths_toBe, angles_f, opt_geom, halogen_positions, neighbours, graph_faces = read_unfolding("../data/unfolding_graph/C60data.h5")
#halogene_positions = np.array([20, 27, 34])
from Unfolding_new import Unfolding
from functions_folding import *
from interpolation import interpolated_values
import pandas as pd
import h5py

f = open("../data/header/header_cam-b3lyp", "r")
header_cam_b3lyp = f.read()
f.close()

In [2]:
def three_dist(unfolding):
    dist_a, dist_b, dist_c = np.linalg.norm(C60.vertex_coords[15] - C60.vertex_coords[35]) , np.linalg.norm(C60.vertex_coords[21] - C60.vertex_coords[22]), np.linalg.norm(C60.vertex_coords[28] - C60.vertex_coords[29])  
    mean_dist = np.mean([dist_a,dist_b,dist_c])
    return np.round(mean_dist,3)

In [3]:
def sample_unfolding(unfolding, title_body="../C60_hinge_0_B3LYP", header=header_cam_b3lyp , start=1.8, end=1.5, step = 0.05, epsilon=0.001,halogen=0):
    sample_r = np.flip(np.arange(end, start, step))
    ### Set the atoms that should be frozen ###
    freeze_list = [15,21,22,28,29,35]
    ### Set the carbon atoms where the halogens should be placed ###
    atoms = [35, 15, 21, 22, 28, 29, 51, 19, 55, 26, 59, 33, 34, 36, 20, 40, 27, 44]
    dist = three_dist(unfolding)
    title = "../" + title_body + "_" + str(dist) +"A"
    write_gaussfile_restart(unfolding, header, freeze=True, connectivity=False, writeFile=True, rotate=True, atoms=atoms, filename = title , freezelist = freeze_list, halogen=halogen, interpolated_angles=interpolated_values)
    for _ in range(unfolding.num_of_steps):
        unfolding.close_unfolding()
        dist = three_dist(unfolding)
        if np.linalg.norm(dist - sample_r[0]) < epsilon:
            sample_r = np.delete(sample_r,0)
            title = title_body + "_" + str(dist) +"A"
            write_gaussfile_restart(unfolding, header, freeze=True, connectivity=False, writeFile=True, rotate=True, atoms=atoms, filename = title , freezelist = freeze_list, halogen=halogen, interpolated_angles=interpolated_values)
            if len(sample_r) == 0:
                return

In [4]:
### Init unfolding and set the number of steps in which the angles are to be closed ###
C60 = Unfolding(dual_unfolding, graph_unfolding_faces, graph_faces, graph_unfolding, neighbours, halogen_positions=halogen_positions, root_node=0, bonds_toBe=bonds_toBe, angles_f=angles_f)
C60.num_of_steps = 20000

In [5]:
sample_unfolding(C60,halogen=1)